In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
import os
import cv2

video_frame_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Resit/sequences/*/')
video_frame_files.sort()
consecutive_frame_pairs_all = []
print(len(video_frame_files))
print(video_frame_files)

62
['/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/animal/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ants1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bag/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ball2/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/ball3/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/basketball/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/birds1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/birds2/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bolt1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/book/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/bubble/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/butterfly/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/car1/', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/conduction1/', '/co

In [3]:
for i in range(5):
    color_file = os.path.join(video_frame_files[i], 'color')
    dosyalar = os.listdir(color_file)
    dosyalar.sort()
    jpg_dosyalari = [dosya for dosya in dosyalar if dosya.endswith(".jpg")]
    jpg_dosyalari.sort()

    consecutive_frame_pairs = []
    #for i in range(len(jpg_dosyalari) - 1):
    for i in range(len(jpg_dosyalari)):
         frame_path = os.path.join(color_file, jpg_dosyalari[i])

         frame = cv2.imread(frame_path, cv2.COLOR_BGR2RGB)  # Görüntüyü okuyun
         #second_frame = cv2.imread(second_frame_path)  # İkinci görüntüyü okuyun

        #consecutive_frame_pairs.append((frame, second_frame))
         consecutive_frame_pairs.append(frame)
    print(len(consecutive_frame_pairs))

    consecutive_frame_pairs_all.append(consecutive_frame_pairs)

100
100
325
311
41


In [4]:
print(len(consecutive_frame_pairs_all))

5


In [5]:
import numpy as np
import torch


bounding_box_folder = '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/*/groundtruth.txt'  # Bounding box dosyalarının bulunduğu klasör
bounding_box_datas = []  # Bounding box bilgilerini içeren frame çiftleri listesi

ground_truth_files = glob.glob(bounding_box_folder)
ground_truth_files.sort()
#print(ground_truth_files)
for file in range(5):

    with open(ground_truth_files[file], 'r') as file:
       bounding_box_lines = file.readlines()
       new_box=[]
       for bounding_box in bounding_box_lines:
          bbox_values = bounding_box.strip().split(',')

          # Bounding box değerlerini float'a dönüştürün
          bbox_values = [float(value) for value in bbox_values]
          resize_ratio=512/512
          # Bounding box koordinatlarını boyutlandırma oranıyla çarpın
          resized_bounding_box =[value * resize_ratio for value in bbox_values]
          tensor_bounding_box = torch.tensor(resized_bounding_box)
          #print(resized_bounding_box)
          new_box.append(tensor_bounding_box)
       bounding_box_datas.append(new_box)
       #print(len(new_box))

       #print(len(bounding_box_lines))


#print(bounding_box_datas[0])
print(len(bounding_box_datas))


5


In [6]:
print(bounding_box_datas[0][0])


tensor([549., 249.,  23.,  57.])


In [7]:

import cv2
import torchvision.transforms as transforms

# Hedef boyut
target_size = (512, 512)

# Transform işlemlerini tanımlayın
transform = transforms.Resize(target_size)

# Görüntüleri boyutlandırın
resized_images = []
for images in consecutive_frame_pairs_all:
  new_images=[]
  for image in images:
    pil_image = transforms.ToPILImage()(image)  # Görüntüyü PIL.Image formatına dönüştürün
    resized_image = transform(pil_image)
    resized_image = transforms.ToTensor()(resized_image)  # Görüntüyü Tensor formatına dönüştürün
    new_images.append(resized_image)
  resized_images.append(new_images)

print(len(resized_images))
consecutive_frame_pairs_all=[]


5


In [8]:
consecutive_frame_2 = []
consecutive_frame_1 = []

for consecutive_frame_pairs in resized_images:
    consecutive_frame_pairs_2= consecutive_frame_pairs[1:]
    consecutive_frame_2.append(consecutive_frame_pairs_2)

for consecutive_frame_pairs_ in resized_images:
    consecutive_frame_pairs_1= consecutive_frame_pairs_[:-1]
    consecutive_frame_1.append(consecutive_frame_pairs_1)

print(consecutive_frame_1[0][0].shape)


torch.Size([3, 512, 512])


In [9]:

bounding_box_datas_2 = []
bounding_box_datas_1 = []

for bounding_box_data in bounding_box_datas:
    bounding_box_data_2= bounding_box_data[1:]
    bounding_box_datas_2.append(bounding_box_data_2)

for bounding_box_data_ in bounding_box_datas:
    bounding_box_datas_= bounding_box_data_[:-1]
    bounding_box_datas_1.append(bounding_box_datas_)

print(len(bounding_box_datas_1[0]))
print(len(bounding_box_datas_2[0]))
print(bounding_box_datas_1[0])
print(bounding_box_datas_2[0])

99
99
[tensor([549., 249.,  23.,  57.]), tensor([553., 242.,  24.,  40.]), tensor([558., 241.,  21.,  37.]), tensor([557., 244.,  27.,  42.]), tensor([558., 255.,  25.,  50.]), tensor([551., 270.,  29.,  36.]), tensor([537., 275.,  33.,  36.]), tensor([524., 280.,  33.,  34.]), tensor([509., 280.,  33.,  35.]), tensor([496., 274.,  37.,  37.]), tensor([494., 275.,  33.,  32.]), tensor([487., 273.,  35.,  34.]), tensor([479., 275.,  42.,  34.]), tensor([487., 275.,  27.,  32.]), tensor([495., 283.,  15.,  23.]), tensor([0., 0., 0., 0.]), tensor([454., 284.,  18.,  26.]), tensor([441., 266.,  36.,  43.]), tensor([422., 255.,  58.,  55.]), tensor([400., 242.,  58.,  47.]), tensor([390., 250.,  57.,  41.]), tensor([373., 256.,  64.,  54.]), tensor([371., 269.,  64.,  51.]), tensor([368., 293.,  65.,  25.]), tensor([359., 290.,  72.,  28.]), tensor([330., 288.,  77.,  39.]), tensor([314., 294.,  70.,  43.]), tensor([292., 292.,  78.,  47.]), tensor([270., 291.,  62.,  44.]), tensor([247., 2

In [10]:
import torch


frame1_tensor = []
for frame_array in consecutive_frame_1:
    for frame in frame_array:
        frame1_tensor.append(frame)
        #print(type(frame))
print(len(frame1_tensor))
print(frame1_tensor[0].shape)



frame2_tensor = []
for frame_array in consecutive_frame_2:
    for frame in frame_array:
        frame2_tensor.append(frame)
print(len(frame2_tensor))




bounding_box1_tensor = []
for bounding_box_array in bounding_box_datas_1:
  for bounding_box in bounding_box_array:
    bounding_box1_tensor.append(bounding_box)
print(len(bounding_box1_tensor))



bounding_box2_tensor = []
for bounding_box_array in bounding_box_datas_2:
  for bounding_box in bounding_box_array:
    bounding_box2_tensor.append(bounding_box)
print(len(bounding_box2_tensor))

872
torch.Size([3, 512, 512])
872
872
872


In [11]:
def crop_frame(frame, bbox):
    # Bounding box değerlerini alalım
    x, y, width, height = bbox.squeeze().tolist()

    # Orta noktayı hesaplayalım
    center_x = x + (width / 2)
    center_y = y + (height / 2)

    # Büyütülmüş bbox boyutlarını hesaplayalım
    enlarged_width = width * 2
    enlarged_height = height * 2

    # Kırpılmış çerçeveyi hesaplayalım
    cropped_frame = frame[:,:,int(center_y - (enlarged_height / 2)):int(center_y + (enlarged_height / 2)),
                          int(center_x - (enlarged_width / 2)):int(center_x + (enlarged_width / 2))]


    #resize_transform = transforms.Resize((224, 224))
    #cropped_frame = resize_transform(cropped_frame)
    # Kanal boyutunu kontrol et
    #assert cropped_frame.shape[2] == 3, "The cropped frame should have 3 channels."

    return cropped_frame


In [44]:
print(len(frame1_tensor))

  #print(frame1_tensor[i].shape)
  #print(frame2_tensor[i].shape)
#print(frame2_tensor[0])

print(bounding_box1_tensor[0])
print(len(bounding_box2_tensor))

872
tensor([308.5714, 139.9532,  12.9274,  32.0375])
872


In [12]:
frame1_tensor = torch.stack(frame1_tensor)
frame2_tensor = torch.stack(frame2_tensor)
bounding_box1_tensor = torch.stack(bounding_box1_tensor)
bounding_box2_tensor = torch.stack(bounding_box2_tensor)


In [13]:
import torch
from torchvision.models import resnet34
import torch.nn as nn


# Install the pre-trained ResNet-34 model
model = resnet34(pretrained=True)

#num_features = model.fc.in_features
#model.fc = torch.nn.Linear(num_features*2, 4)  # Adjust the output size to 4 for the bounding box coordinates




# Create training installer
#dataset = torch.utils.data.TensorDataset(frame1_tensor, frame2_tensor, bounding_box1_tensor, bounding_box2_tensor)
dataset = torch.utils.data.TensorDataset(frame1_tensor, frame2_tensor, bounding_box1_tensor, bounding_box2_tensor)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

print(dataset[0][0].shape)
print(dataset[1][0].shape)
# Train Model
criterion = torch.nn.MSELoss()  # Kayıp fonksiyonu olarak MSE kullanıyoruz
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # SGD optimizerini kullanıyoruz

epoch=10 #train for 10 epochs
for epoch in range(epoch):
    running_loss = 0.0

    for frame1, frame2, bounding_box1, bounding_box2 in loader:
        optimizer.zero_grad()

        resized_transform = transforms.Resize((224, 224))

        # Clipping and feature extraction for the first frame

        #cropped_frame_1 = crop_frame(frame1.squeeze(0), bounding_box1)
        cropped_frame_1 = crop_frame(frame1, bounding_box1)  # Crop the first frame
        if cropped_frame_1.size(0) > 0 and cropped_frame_1.size(1) > 0:
            resized_transform = transforms.Resize((224, 224))
            resized_frame_1 = resized_transform(cropped_frame_1)
            resized_frame_1 = resized_frame_1.unsqueeze(0)  # Add batch dimension
            features_1 = model(resized_frame_1)
            features_1 = features_1.view(features_1.size(0), -1)

        #print(cropped_frame_1.permute(2, 0, 1).unsqueeze(0))
        #features_1 = model(cropped_frame_1)  # Feature extraction

        # Clipping and feature extraction for the first frame
        cropped_frame_2 = crop_frame(frame2 ,bounding_box1)  # Crop the second frame

        if cropped_frame_2.size(0) > 0 and cropped_frame_2.size(1) > 0:
            resized_frame_2 = resized_transform(cropped_frame_2)
            resized_frame_2 = resized_frame_2.unsqueeze(0)  # Add batch dimension
            features_2 = model(resized_frame_2)
            features_2 = features_2.view(features_2.size(0), -1)
        #features_2 = model(cropped_frame_2)  # Feature extraction
        #print(features_2.shape)

        # Combining feature vectors
        combined_features = torch.cat((features_1, features_2), dim=1)


        print(combined_features.shape)
        # Predictions
        #predictions =model.fc(combined_features)
        predictions = model.fc(combined_features.view(combined_features.size(0), -1))
        print(predictions)

        # lass calculation and back propagation
        loss = criterion(predictions, bounding_box2)
        loss.backward()


        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 325MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomRes

torch.Size([3, 512, 512])
torch.Size([3, 512, 512])


RuntimeError: ignored

In [42]:
cropped_frame1=[]
for i in range(10):
  cropped_frames=[]
  for j in range(len(frame2_path_all[i])):
    ground_truth=np.array(list(map(float, bounding_box1_all[i][j].split(','))))
    enlarged_bbox = ground_truth * 2  # Bounding box'u 2 kat büyütün
    bbox_center = (ground_truth[0] + ground_truth[2] / 2, ground_truth[1] + ground_truth[3] / 2)  # Bounding box'un merkezini hesaplayın
    bbox_size = (enlarged_bbox[2], enlarged_bbox[3])  # Bounding box'un boyutunu alın
    x1 = int(bbox_center[0] - bbox_size[0] / 2)  # Bounding box'un sol üst köşesinin x koordinatını hesaplayın
    y1 = int(bbox_center[1] - bbox_size[1] / 2)  # Bounding box'un sol üst köşesinin y koordinatını hesaplayın
    x2 = int(bbox_center[0] + bbox_size[0] / 2)  # Bounding box'un sağ alt köşesinin x koordinatını hesaplayın
    y2 = int(bbox_center[1] + bbox_size[1] / 2)  # Bounding box'un sağ alt köşesinin y koordinatını hesaplayın

    cropped_frame = frame[y1:y2, x1:x2]
    cropped_frames.append(cropped_frame)
  print(len(cropped_frames))
  cropped_frame1.append(cropped_frames)
print(len(cropped_frame1))



99
99
324
310
40
170
724
338
79
349
10


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim

# 1. Eğitim veri yükleyicisi oluşturma
# TODO: Eğitim veri yükleyicisini hazırlayın

# 2. Frame kırpması
def crop_frame(frame, bbox):
    x1 = int(bbox[0] - bbox[2] / 2)
    y1 = int(bbox[1] - bbox[3] / 2)
    x2 = int(bbox[0] + bbox[2] / 2)
    y2 = int(bbox[1] + bbox[3] / 2)
    return frame[y1:y2, x1:x2]

# 3. Pre-trained ResNet-34 modelinin yüklenmesi
model = models.resnet34(pretrained=True)
model.eval()

# 4. Özellik çıkarma
def extract_features(image, model):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        features = model(input_batch)
    return features

# 5. Özellik birleştirme
class FeatureConcatenation(nn.Module):
    def __init__(self):
        super(FeatureConcatenation, self).__init__()

    def forward(self, features1, features2):
        return torch.cat((features1, features2), dim=1)

# 6. Tam bağlantılı katmanlar ve kayıp fonksiyonu
class BoundingBoxPrediction(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BoundingBoxPrediction, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 4)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

# Hyperparametreler
input_size = 2 * model.fc.in_features
hidden_size = 512
learning_rate = 0.001
num_epochs = 10

# Model oluşturma
concatenation = FeatureConcatenation()
bounding_box_prediction = BoundingBoxPrediction(input_size, hidden_size)

# Optimizer ve kayıp fonksiyonu
optimizer = optim.Adam(list(concatenation.parameters()) + list(bounding_box_prediction.parameters()), lr=learning_rate)
criterion = nn.MSELoss()

# Eğitim döngüsü
for epoch in range(num_epochs):
    for frames in dataloader:  # 1. adımda oluşturulan eğitim veri yükleyicisini kullanarak frame çiftlerini alın
        t0, t1 = frames  # Örnek olarak sadece iki frame kullanıldı

        # 2. Frame kırpması
        cropped_frame1 = crop_frame(t0, padded_ground_truth_bbox)  # İlk frame'in kırpılması
        cropped_frame2 = crop_frame(t1, search_region_bbox)  # İkinci frame'in kırpılması

        # 3. Özellik çıkarma
        features1 = extract_features(cropped_frame1, model)
        features2 = extract_features(cropped_frame2, model)

        # 4. Özellik birleştirme
        concatenated_features = concatenation(features1, features2)

        # 5. Tahmin ve kayıp hesaplama
        predicted_bbox = bounding_box_prediction(concatenated_features)
        loss = criterion(predicted_bbox, ground_truth_bbox)

        # Geriye yayılım ve optimizasyon
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

# Eğitimi tamamladıktan sonra, modeliniz eğitilmiş olacak ve tahminler yapabilirsiniz.


In [15]:
import cv2

frame1_cropped=[]
print(len(bounding_box1_all))
for frame1_path, bounding_box1 in zip(frame1_path_all, bounding_box1_all):

  #print(frame1_path)
  frame1_img=[]
  for frame1_path_ , bounding_box1_ in zip(frame1_path, bounding_box1):
     frame1= cv2.imread(frame1_path_)
     #print(frame1)
     #print(bounding_box1_)
     x1, y1, w1, h1 = map(float, bounding_box1_.split(','))
     frame_height, frame_width, _ = frame1.shape  # Kare boyutunu alın
     center_x = x1 + w1 / 2  # Bounding box'un merkezinin x koordinatı
     center_y = y1 + h1 / 2  # Bounding box'un merkezinin y koordinatı
     enlarged_width = int(w1 * 2)  # Genişlik 2 kat büyütülecek
     enlarged_height = int(h1 * 2)  # Yükseklik 2 kat büyütülecek
     x_crop = int(center_x - enlarged_width / 2)  # Kırpma için sol üst köşenin x koordinatı
     y_crop = int(center_y - enlarged_height / 2)  # Kırpma için sol üst köşenin y koordinatı

     # Kırpma işlemini gerçekleştirin
     cropped_frame = frame1[y_crop:y_crop + enlarged_height, x_crop:x_crop + enlarged_width]
     frame1_img.append(cropped_frame)
  frame1_cropped.append(frame1_img)
  print(len(frame1_img))







# İlk frame'i yükleyin
#frame1 = cv2.imread(frame1_path)

# Ground-truth sınırlayıcı kutudan koordinatları alın
#x1, y1, w1, h1 = bounding_box1

# Sınırlayıcı kutuyu 2 kat büyütün
#x1 -= int(w1 / 2)
#y1 -= int(h1 / 2)
#w1 *= 2
#h1 *= 2

# Büyütülmüş sınırlayıcı kutuyla ilk frame'i kırpın
#cropped_frame1 = frame1[y1:y1+h1, x1:x1+w1]


10


TypeError: ignored

In [ ]:
import cv2
def read_img( path):
  new_arr_img=[]
  for img in path:
    img_arr=cv2.imread(img)
    new_arr_img.append(img_arr)

  return new_arr_img



In [ ]:
frame1_img_all=[]

In [ ]:
import cv2

for i in range(0,15):
  print(frame1_path_all[i])
  frame1_img_all.append(read_img(frame1_path))


['/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000001.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000002.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000003.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000004.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000005.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000006.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000007.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000008.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000009.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000010.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/color/00000011.jpg', '/content/drive/MyDrive/Colab Notebooks/Resit/sequences/agility/

In [ ]:
import cv2

for frame1_path in range(len(frame1_path_all)/4,len(frame1_path_all)/2):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
import cv2

for frame1_path in range(len(frame1_path_all)/2,3*(len(frame1_path_all)/4)):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
import cv2

for frame1_path in range(3*(len(frame1_path_all)/4),len(frame1_path_all)):
  print(frame1_path[0])
  frame1_img_all.append(read_img(frame1_path))

In [ ]:
# İkinci frame'i yükleyin
frame2 = cv2.imread(frame2_path)

# Önceki frame'in sınırlayıcı kutusundan koordinatları alın
x2, y2, w2, h2 = bounding_box2

# Sınırlayıcı kutuyu 2 kat büyütün
x2 -= int(w2 / 2)
y2 -= int(h2 / 2)
w2 *= 2
h2 *= 2

# Büyütülmüş sınırlayıcı kutuyla ikinci frame'i kırpın
cropped_frame2 = frame2[y2:y2+h2, x2:x2+w2]
